In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from toolkit.common.constants import *
from toolkit.cars.car_configuration import Car
from toolkit.lap.track import *
from toolkit.lapsim.multi_sim import MultiSim

In [ ]:
tracks = []
v_average = 15
mu_corr = 0.65 # mu correction factor, a value of 1 means the road is sandpaper and the actual value should be something lower but im kinda just setting this value to overfit atm
target = 0.001
tr = 600 * FTLB_TO_NM

In [ ]:
# here we make a function that generates a car with a given roll stiffness distribution and chassis roll stiffness
# the first and second arguments take the values of the first and second sweep variables that you give to run_sim
def gen_car_lltd(rsd, trs):
    total_roll_stiffness = trs * FTLB_TO_NM
    car = Car()
    car.z_f = 0.13 * IN_TO_M
    car.z_r = 1.0 * IN_TO_M
    car.k_f = total_roll_stiffness * rsd
    car.k_r = total_roll_stiffness * (1-rsd)
    car.k_c = tr
    car.description = f"rsd:{rsd:.2f} trs:{trs:.2f}"
    return car

# do a sweep of roll stiffness distribution and chassis roll stiffness
rsd_range = np.linspace(0.1, 0.8, 50)
# k_c_r = np.linspace(100, 10000, 50)
trs_r = np.geomspace(250, 1750, 25)
sim = MultiSim(tracks, gen_car_lltd, rsd_range, trs_r, "Roll Stiffness Distribution", "Suspension Roll Stiffness (lbf/deg)")

In [ ]:
sim.run_lltd()

In [ ]:
fig = go.Figure()
fig.add_contour(x=rsd_range, y=((tr)/(trs_r * FTLB_TO_NM)), z=np.abs(sim.lltd_diff) * 100, colorbar=dict(title="LLTD error %"), colorscale='Viridis', contours=dict(
            coloring ='heatmap',
            showlabels = True, # show labels on contours
            labelfont = dict( # label font properties
                size = 12,
                color = 'white',)),
            line_smoothing=0.85)
# fig.add_scatter(x=[0.2, 0.8], y=[1.5, 1.5], mode="lines", name="Target", line=dict(color="red", dash="dash"))
fig.update_layout(title="LLTD Error", xaxis_title="λ, Roll Stiffness Distribution (% Front)", yaxis_title="μ, Kc/(Kf+Kr)", height=768, width=1024)
# make the y axis log scale
fig.update_yaxes(type="log")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_contour(x=rsd_range, y=(tr/(trs_r * FTLB_TO_NM)), z=sim.lltd, colorbar=dict(title="LLTD error %"), colorscale='Viridis', contours=dict(
            coloring ='heatmap',
            showlabels = True, # show labels on contours
            labelfont = dict( # label font properties
                size = 12,
                color = 'white',)),
            line_smoothing=0.85)
# mu = tr / (trs_r * FTLB_TO_NM)
# fig.add_scatter(x=[0.2, 0.8], y=[mu, mu], mode="lines", name="Target", line=dict(color="red", dash="dash"))
fig.update_layout(title="LLTD Error", xaxis_title="λ, Roll Stiffness Distribution (% Front)", yaxis_title="μ, Kc/(Kf+Kr)", height=768, width=1024)
# make the y axis log scale
fig.update_yaxes(type="log")
fig.show()

In [ ]:
car = gen_car_lltd(0.5, 100)
setting_names = ["D", "S-S", "S-M", "M-M", "M-H", "H-H"]
front_bellcrank_ratios = np.array([0.0, 0.28843, 0.32397, 0.35952, 0.38328, 0.40705])
rear_bellcrank_ratios = np.array([0.0, 0.26927, 0.31020, 0.35112, 0.39029, 0.42945])
front_arb_rate, rear_arb_rate = 3.39 * FTLB_TO_NM, 7.86 * FTLB_TO_NM
lever_arm_length = 2.5 * IN_TO_M
front_arb_settings = front_arb_rate * (front_bellcrank_ratios**2) * (car.front_track**2) / (lever_arm_length**2)
rear_arb_settings = rear_arb_rate * (rear_bellcrank_ratios**2) * (car.rear_track**2) / (lever_arm_length**2)
front_ride_roll_stiffness, rear_ride_roll_stiffness = 172 * FTLB_TO_NM, 172 * FTLB_TO_NM
front_roll_stiffness, rear_roll_stiffness = front_arb_settings + front_ride_roll_stiffness, rear_arb_settings + rear_ride_roll_stiffness

frs_v, rrs_v = np.meshgrid(front_roll_stiffness, rear_roll_stiffness)
trs_v = frs_v + rrs_v
rsd_v = frs_v / trs_v

In [ ]:
fig = go.Figure()
fig.add_contour(x=rsd_range, y=trs_r, z=sim.lltd, colorbar=dict(title="LLTD (% Front)"), colorscale='Viridis', contours=dict(
            coloring ='heatmap',
            showlabels = True, # show labels on contours
            labelfont = dict( # label font properties
                size = 12,
                color = 'white',)),
            line_smoothing=0.85)
# mu = tr / (trs_r * FTLB_TO_NM)
# fig.add_scatter(x=[0.2, 0.8], y=[mu, mu], mode="lines", name="Target", line=dict(color="red", dash="dash"))

# plot all arb settings
front_settings = setting_names * len(front_arb_settings)
rear_settings = np.repeat(setting_names, len(front_arb_settings))
arb_labels = [f"F:{frs:.0f}<br>{fsn}<br>R:{rrs:.0f}<br>{rsn}" for frs, rrs, fsn, rsn in zip(frs_v.flatten(), rrs_v.flatten(), front_settings, rear_settings)]
fig.add_scatter(x=rsd_v.flatten(), y=trs_v.flatten(), mode="markers", name="ARB Settings", marker=dict(color="red", size=4), text=arb_labels, showlegend=False)
# add labels to the right of each of the markers
for i, arb_label in enumerate(arb_labels):
    fig.add_annotation(x=rsd_v.flatten()[i], y=trs_v.flatten()[i], text=arb_label, showarrow=False, xshift=12, yshift=-4, font=dict(size=6))


fig.update_layout(title="LLTD", xaxis_title="λ, Roll Stiffness Distribution (% Front)", yaxis_title="Total Roll Stiffness (lb-ft/deg)", height=768, width=1024)
# make the y axis log scale
fig.show()